In [3]:
import time
import serial

# Arduino configuration
arduino_port = "/dev/cu.usbserial-A50285BI"  # Replace with the correct port name for your Arduino
baud_rate = 115200

# Create a serial connection with the Arduino
def create_serial_connection(port, baud_rate):
    try:
        ser = serial.Serial(port, baud_rate, timeout=1)
        time.sleep(2)  # Wait for the connection to establish
        return ser
    except serial.SerialException as e:
        print(f"Failed to connect to Arduino: {e}")
        exit(1)

# Function to send serial command to the Arduino
def send_command(ser, command):
    try:
        ser.write(command.encode())  # Send command to Arduino
        time.sleep(1)  # Wait for Arduino to process the command
        response = ser.readline().decode().strip()  # Read response
        print(f"Arduino Response: {response}")
    except serial.SerialException as e:
        print(f"Serial Exception: {e}")

def main():
    # Create a serial connection with the Arduino
    ser = create_serial_connection(arduino_port, baud_rate)

    # Example usage of the send_command function
    command_to_send = "A"  # Replace with the actual command you want to send (a letter or number)
    send_command(ser, command_to_send)

    # Close the serial connection before exiting the script
    ser.close()

if __name__ == "__main__":
    main()
    

Failed to connect to Arduino: [Errno 2] could not open port /dev/cu.usbserial-A50285BI: [Errno 2] No such file or directory: '/dev/cu.usbserial-A50285BI'


AttributeError: 'NoneType' object has no attribute 'write'

In [ ]:
import casadi as ca

# Define CasADi MX symbols
theta1 = ca.MX.sym('theta1')
theta2 = ca.MX.sym('theta2')
l1 = ca.MX.sym('l1')
l2 = ca.MX.sym('l2')

# Define the forward kinematics equations for CasADi
xe_ca = l1 * ca.cos(theta1) + l2 * ca.cos(theta1 + theta2)
ye_ca = l1 * ca.sin(theta1) + l2 * ca.sin(theta1 + theta2)

# Combine into a vector
end_effector_ca = ca.vertcat(xe_ca, ye_ca)

# Compute the Jacobian matrix using CasADi
jacobian_ca = ca.jacobian(end_effector_ca, ca.vertcat(theta1, theta2))

# Create a CasADi function for the Jacobian 
J_func = ca.Function('J_func', [theta1, theta2, l1, l2], [jacobian_ca])

# Compute the inverse of the Jacobian matrix using CasADi
jacobian_inv_ca = ca.inv(jacobian_ca)

# Create a CasADi function for the Jacobian inverse
jacobian_inv_func = ca.Function('jacobian_inv_ca', [theta1, theta2, l1, l2], [jacobian_inv_ca])

# Example usage with CasADi
theta1_degree = 0.01      # Initial values
theta2_degree = 179.9      # Initial values

#converted to radian
theta1_val = ((theta1_degree * ca.pi) / 180)
theta2_val = ((theta2_degree * ca.pi) / 180)
l1_val = 14.0
l2_val = 15.0


# Request user input for the next position
xe_2 = float(input("\nEnter the Target x-coordinate of the next position: "))
ye_2 = float(input("Enter the Target y-coordinate of the next position: "))
new_position = ca.vertcat(xe_2, ye_2)

# Iterative loop to update position
tolerance = 1e-7  # Define a tolerance for convergence
max_iterations = 200  # Define a maximum number of iterations
iteration = 0

while iteration < max_iterations:
    # Compute the current end effector position
    current_position = ca.vertcat(
        l1_val * ca.cos(theta1_val) + l2_val * ca.cos(theta1_val + theta2_val),
        l1_val * ca.sin(theta1_val) + l2_val * ca.sin(theta1_val + theta2_val)
    )
    
    # Calculate the difference between the current and target positions
    dx = new_position - current_position
    
    # Covert Inverse Jacobian function into a Dense Matrix or DM
    jacobian_inv_val = jacobian_inv_func(theta1_val, theta2_val, l1_val, l2_val)
    
    # Calculate the change in joint angles
    dq = jacobian_inv_val @ dx
    
    # Update joint angles
    theta1_val = theta1_val + dq[0]
    theta2_val = theta2_val + dq[1]
    
    # Check for convergence
    if ca.norm_2(dx) < tolerance:
        print("\nConvergence achieved.")
        break

    iteration += 1

if iteration >= max_iterations:
    print("\nMax iterations reached. Convergence not achieved.")

print(f"\nFinal joint angles: {theta1_val, theta2_val}")
print(f"Final end effector position: {new_position}")


In [2]:
import time
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)